In [25]:
from IPython.display import Image
Image("postal_codes_canada_wikipedia.png")

In [26]:
import requests
import lxml.html as lh
import pandas as pd

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [27]:
page = requests.get(url)
doc = lh.fromstring(page.content)

In [28]:
tr_elements = doc.xpath('//tr')

In [29]:
type(tr_elements), len(tr_elements)

(list, 294)

In [30]:
print(tr_elements[0].text_content())
print(tr_elements[1].text_content())


Postcode
Borough
Neighbourhood


M1A
Not assigned
Not assigned



In [31]:
data = []
for tr in tr_elements:
    tr_content = tr.text_content().split("\n")
    row = [item for item in tr_content if str(item).strip() != ""]
    if len(row) == 3:
        data.append(row)

In [32]:
df = pd.DataFrame(data)
df.columns = df.loc[0].values.tolist()
df.drop(df.index[0], inplace=True)
df = df[df["Borough"] != "Not assigned"]
df.reset_index(inplace=True, drop=True)
df.head(3)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront


In [33]:
df[df["Neighbourhood"] == "Not assigned"] 

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Not assigned


In [34]:
pd.options.mode.chained_assignment = None  # default='warn'

postcodes = df["Postcode"].unique()
neighbourhoods = []
boroughs = []
for postcode in postcodes:
    boro = df[df["Postcode"] == postcode]["Borough"].unique().tolist()
    if len(boro) != 1:
        raise ValueError("There should only be one Borough")
    boroughs.append(boro[0])
    
    hood = df[df["Postcode"] == postcode]["Neighbourhood"].unique().tolist()
    neighbourhoods.append(", ".join(hood))

dft = pd.DataFrame({"Postcode": postcodes, "Borough": boroughs, "Neighbourhood": neighbourhoods})

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
# specifially, this turns up for M7A, Queen's Park above
borough_tocopy = dft.loc[dft["Neighbourhood"]=="Not assigned"]["Borough"].values.tolist()
print(borough_tocopy)
dft.loc[dft["Neighbourhood"]=="Not assigned", "Neighbourhood"] = borough_tocopy

# rename the postal code column
dft.rename(columns={'Postcode':'PostCode'}, inplace=True)
dft

["Queen's Park"]


,Borough,Neighbourhood,PostCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Harbourfront, Regent Park",M5A
3,North York,"Lawrence Heights, Lawrence Manor",M6A
4,Queen's Park,Queen's Park,M7A
5,Etobicoke,Islington Avenue,M9A
6,Scarborough,"Rouge, Malvern",M1B
7,North York,Don Mills North,M3B
8,East York,"Woodbine Gardens, Parkview Hill",M4B
9,Downtown Toronto,"Ryerson, Garden District",M5B


In [35]:
dft[dft["PostCode"] == "M5A"]

,Borough,Neighbourhood,PostCode
2,Downtown Toronto,"Harbourfront, Regent Park",M5A


In [36]:
dft[dft["PostCode"] == "M7A"] 

,Borough,Neighbourhood,PostCode
4,Queen's Park,Queen's Park,M7A


In [37]:
dft.shape 

(103, 3)

In [ ]:
import geocoder

In [ ]:
# initialize your variable to None
postal_code = "M5G"
lat_lng_coords = None

# loop until you get the coordinates
counter = 0
while(lat_lng_coords is None):
    counter += 1
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    print("Attempt %d, lat_lng_coords = %s" % (counter, str(lat_lng_coords)))
    if counter >= 10:
        break

In [ ]:
df_coords = pd.read_csv("Geospatial_Coordinates.csv")
df_coords.rename(columns={'Postal Code':'PostCode'}, inplace=True)
df_coords.set_index("PostCode", inplace=True)
df_coords.head(3)

In [ ]:
lat = []
lng = []
for pc in dft["PostCode"].values:
    lat.append(df_coords.at[pc, "Latitude"])
    lng.append(df_coords.at[pc, "Longitude"])

In [ ]:
dft["Latitude"] = lat
dft["Longitude"] = lng

dft

In [ ]:
dft[dft["PostCode"] == "M5G"]

In [ ]:
dft[dft["PostCode"] == "M5A"]

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

In [ ]:
# https://www.latlong.net/place/toronto-on-canada-27230.html
tlat = 43.651070
tlng = -79.347015

In [ ]:
dft.columns

In [ ]:
map_toronto = folium.Map(location=[tlat, tlng], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dft['Latitude'], dft['Longitude'], dft['Borough'], dft['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [ ]:
CLIENT_ID = '2NOYPO3T3EULZM4SMI5Q4B00SBBLEYD3LWNYARIQNGQMIPWB'     # your Foursquare ID
CLIENT_SECRET = 'RHT5F4BM1TSDIMA33YRGST1POCB3VZQ2UNNQR2ZC3245Q4UI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
dff = dft[dft["Borough"].str.contains("Toronto")]
dff.head()

In [ ]:
len(dff.index)

In [ ]:
lats = dff["Latitude"].values.tolist()
lngs = dff["Longitude"].values.tolist()
len(lats), len(lngs)

In [ ]:
map_toronto = folium.Map(location=[tlat, tlng], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dff['Latitude'], dff['Longitude'], dff['Borough'], dff['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [ ]:
def make_venues_url(lat, lng, radius=100, limit=500):
    url = "https://api.foursquare.com/v2/venues/explore?" \
        + "client_id={}".format(CLIENT_ID) \
        + "&client_secret={}".format(CLIENT_SECRET) \
        + "&v={}".format(VERSION) \
        + "&ll={},{}".format(lat, lng) \
        + "&radius={}".format(radius) \
        + "&limit={}".format(limit)
    return url

url = make_venues_url(lats[0], lngs[0])
url

In [ ]:
results = requests.get(url).json()

In [ ]:
type(results["response"])

In [ ]:
list(results["response"].keys())

In [ ]:
results["response"]["totalResults"]

In [ ]:
LIMIT=500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
    
    return(nearby_venues)

In [ ]:
dff.columns

In [ ]:
toronto_venues = getNearbyVenues(names=dff['Neighbourhood'],
                                 latitudes=dff['Latitude'],
                                 longitudes=dff['Longitude'])

In [ ]:
len(toronto_venues), type(toronto_venues)

In [ ]:
toronto_venues.head()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [ ]:
sorted(toronto_venues['Venue Category'].unique().tolist())[:10]

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

In [ ]:
toronto_grouped.shape

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
toronto_grouped.columns

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
tlat, tlng

In [ ]:
# create map
map_clusters = folium.Map(location=[tlat, tlng], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
c1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
c1.head()

In [ ]:
c2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
c2.head()

In [ ]:
c3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
c3.head()

In [ ]:
c4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
c4.head()